# Using ```bayesmixedlogit``` and ```bayesmixedlogitwtp``` in Stata

This document presents an overview of the ```bayesmixedlogit``` and ```bayesmixedlogitwtp``` `Stata` packages. It mirrors closely the helpfile obtainable in ```Stata``` (i.e., through ``` help bayesmixedlogit``` or ```help bayesmixedlogitwtp```). Further background for the packages can be found in [Baker(2014)](https://www.stata-journal.com/article.html?article=st0354).



## Description

```bayesmixedlogit``` can be used to fit mixed logit models using Bayesian
methods -- more precisely, ```bayesmixedlogit``` produces draws from the
posterior parameter distribution and then presents summary and other
statistics describing the results of the drawing. Detailed analysis of
the draws is left to the discretion of the user. 


Implementation of `bayesmixedlogit` follows [Train (2009, chap. 12)](https://eml.berkeley.edu/books/choice2.html), and details of how the algorithm works are described
in [Baker(2014)](https://www.stata-journal.com/article.html?article=st0354).  A diffuse prior for the mean values of the random coefficients is assumed, and the prior distribution
    on the covariance matrix of random coefficients is taken to be an identity inverse Wishart.  ```bayesmixedlogit``` uses the
    ```Mata``` routines ```amcmc()``` (if not installed: ```ssc install amcmc```) for adaptive Markov chain Monte Carlo sampling from the
    posterior distribution of individual level coefficients and fixed coefficients.  The data setup for ```bayesmixedlogit``` is
    the same as for ```clogit``` (```ssc install clogit```).  Much of the syntax follows that used by [Hole (2007)](https://www.stata-journal.com/article.html?article=st0133) in development of the command ```mixlogit```.


## Options

**```group```**(_`varname`_) specifies a numeric identifier variable for choice occasions.  **```group```**() is required.

**```identifier```**(_`varname`_) identifies coefficient sets (those observations for which a set of coefficients apply).  Thus, when a person is observed making choices over multiple occasions, one would use **```group```**(_`varname`_) to specify the
        choice occasions, while **`identifier`**(_`varname`_) would identify the person.  **`identifier`**() is required.

**`rand`**(_`varlist`_) specifies independent variables with random coefficients.  The variables immediately following the dependent variable in the syntax are considered to have fixed coefficients (see the examples below).  While a model
can be run without any independent variables with fixed coefficients, at least one random-coefficient independent variable is required for bayesmixedlogit to work.  **```rand```**() is required.

**`draws`**(`#`) specifies the number of draws that are to be taken from the posterior distribution of the parameters.  The default is **`draws`**(`1000`).

**`drawsrandom`**(#) is an advanced option.  The drawing algorithm treats each set of random coefficients as a Gibbs step in sampling from the joint posterior distribution of parameters.  In difficult, large-dimensional problems, it
        might be desirable to let individual Gibbs steps run for more than one draw to achieve better mixing and convergence of the algorithm.

**`drawsfixed`**(#) is a more advanced option.  The drawing algorithm treats fixed coefficients as a Gibbs step in sampling from the joint posterior distribution of parameters.  In difficult, large-dimensional problems, it might be
        desirable to let this step in Gibbs sampling run for more than a single draw.  The default is **`drawsfixed(1)`**.

**`burn`**(#) specifies the length of the burn-in period; the first # draws are discarded upon completion of the algorithm and before further results are computed.

**`thin`**(#) specifies that only every #th draw is to be retained, so if thin(3) is specified, only every third draw is retained.  This option is designed to help ease autocorrelation in the resulting draws, as is the option jumble,
        which randomly mixes draws.  Both options may be applied.

**`araterandom`**(#) specifies the desired acceptance rate for random coefficients and should be a number between zero and one.  Because an adaptive acceptance-rejection method is used to sample random coefficients, by specifying the
desired acceptance rate, the user has some control over adaptation of the algorithm to the problem.  The default is **`araterandom`**(.234).

**`aratefixed`**(#) specifies the desired acceptance rate for fixed coefficients and works in the same fashion as **`araterandom(#)`**.

**`samplerrandom`**(_`string`_) specifies the type of sampler that is to be used when random parameters are drawn.  It may be set to either global or mwg.  The default is **`samplerrandom`**(_`global`_), which means that proposed changes to random
        parameters are drawn all at once.  If `mwg` -- an acronym for "Metropolis within Gibbs" -- is instead chosen, each random parameter is drawn separately as an independent step conditional on other random parameters in a nested
        Gibbs step.  `mwg` might be useful in situations in which initial values are poorly scaled.  The workings of these options are described in greater detail in [Baker(2014)](https://www.stata-journal.com/article.html?article=st0354).

**`samplerfixed`**(_`string`_) specifies the type of sampler that is used when fixed parameters are drawn.  Options are exactly as those described under **`samplerrandom`**(_`string`_).

**`dampparmfixed`**(#) works exactly as option **dampparmrandom**(#) but is applied to drawing fixed parameters.

**`dampparmrandom`**(#) is a parameter that controls how aggressively the proposal distributions for random parameters are adapted as drawing continues.  If the parameter is set close to one, adaptation is aggressive in its early phase
        of trying to achieve the acceptance rate specified in **`araterandom`**(#).  If the parameter is set closer to zero, adaptation is more gradual.

**`from`(_`rowvector`_) specifies a row vector of starting values for all parameters in order.  If these are not specified, starting values are obtained via estimation of a conditional logit model via **`clogit`**.

**`fromvariance`**(_`matrix`_) specifies a matrix of starting values for the random parameters.

**`jumble`** specifies to randomly mix draws.

**`noisy`** specifies that a dot be produced every time a complete pass through the algorithm is finished.  After 50 iterations, a function value `ln_fc(p)` will be produced, which gives the joint log of the value of the posterior choice
        probabilities evaluated at the latest parameters.  While `ln_fc(p)` is not an objective function per se, the author has found that drift in the value of this function indicates that the algorithm has not yet converged or has
        other problems.

**`saving`**(_`filename`_) specifies a location to store the draws from the distribution.  The file will contain just the draws after any burn-in period or thinning of values is applied.

**replace** specifies that an existing file is to be overwritten.

**append** specifies that an existing file is to be appended, which might be useful if multiple runs need to be combined.

**`indsave`**(_`filename`_) specifies a file to which individual-level random parameters are to be saved. More precisely, **`indsave`**(_`filename`_) saves the draws of the individual-level parameters. Caution: For long runs and models with large
        numbers of individuals, specifying this option can cause memory problems. Users should be careful how it is used and consult some of the examples before employing the option.

**`indkeep`**(#) is for use with indsave and specifies that only the last # draws of the individual-level random parameters be kept. This helps avoid excessive memory consumption.

**`indwide`** is for use with indsave and affords the user a degree of control over how individual-level parameters are saved. By default, individual-level parameters are saved in a panel form, meaning that each random parameter draw
        is saved in a row, where draws are marked by the group identifier.  If instead the user would prefer that each row contain draws of each parameter, one could specify the indwide option, which saves all draws in a single row,
        with the first entry of the row being the group identifier.  By analogy with `reshape`, by default draws are saved in "long" format, whereas **`indwide`** stores the draws in "wide" format.

**`replaceind`** functions in the same way as replace, but in reference to the file specified in **`indsave`**.

**`appendind`** functions in the same way as **`append`**, but in reference to the file specified in **`indsave`**.




## Examples

### Example 1

A single random coefficient, one decision per group.  The random parameter acceptance rate is set to 0.4, and a total of 4,000 draws are taken.  The first 1,000 draws are dropped, and then every fifth draw is retained.  Draws are saved as
    `choice_draws.dta`:


In [2]:
webuse choice
describe




Contains data from http://www.stata-press.com/data/r14/choice.dta
  obs:           885                          
 vars:             7                          2 Dec 2014 13:25
 size:         9,735                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
id              int     %9.0g                 
sex             byte    %9.0g      sex        
income          float   %9.0g                 income in thousands
car             byte    %9.0g      nation     nationality of car
size            byte    %9.0g                 
choice          byte    %9.0g                 ID's chosen car
dealer          byte    %9.0g                 number of dealers of each
                                                nationality in ID's city
-------------------------------

In [3]:
bayesmixedlogit choice, rand(dealer) group(id) id(id) ///
     draws(4000) burn(1000) thin(5) arater(.4) saving(choice_draws) replace



Bayesian Mixed Logit Model                         Observations    =       885
                                                   Groups          =       295
Acceptance rates:                                  Choices         =       295
 Fixed coefs              =                        Total draws     =      4000
 Random coefs(ave,min,max)= 0.239, 0.186, 0.285    Burn-in draws   =      1000
                                                   *One of every 5 draws kept
------------------------------------------------------------------------------
      choice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
Random       |
      dealer |   .2150072   .0360706     5.96   0.000     .1441673    .2858471
-------------+----------------------------------------------------------------
Cov_Random   |
  var_dealer |   .1024538   .0322976     3.17   0.002     .0390238    .1658839
---------------------

### Example 2

Fitting a mixed logit model using `bayesmixedlogit`, using the methods as described in [Long and Freese (2006, sec. 7.2.4)](https://www.stata-press.com/books/regression-models-categorical-dependent-variables/long3-brochure.pdf). The data must first be rendered into the correct format, which can be done using the command `case2alt`, which
    is part of the package `spost9_ado`; if not installed, type `net describe spost9_ado` from the `Stata` prompt. The example first arranges the data and then generates and summarizes posterior draws from a mixed logit model. The model uses `bangladesh.dta`, which has information on contraceptive choice by a series of families.  Coefficients of explanatory variables vary at the district level.


In [4]:
webuse choice
describe




Contains data from http://www.stata-press.com/data/r14/choice.dta
  obs:           885                          
 vars:             7                          2 Dec 2014 13:25
 size:         9,735                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
id              int     %9.0g                 
sex             byte    %9.0g      sex        
income          float   %9.0g                 income in thousands
car             byte    %9.0g      nation     nationality of car
size            byte    %9.0g                 
choice          byte    %9.0g                 ID's chosen car
dealer          byte    %9.0g                 number of dealers of each
                                                nationality in ID's city
-------------------------------

In [5]:
webuse bangladesh, clear
case2alt, casevars(urban age) choice(c_use) gen(choice)


(Bangladesh Fertility Survey, 1989)

(note: variable _id used since case() not specified)
(note: variable _altnum used since altnum() not specified)

choice indicated by: choice
case identifier: _id
case-specific interactions: no* yes*


In [6]:
bayesmixedlogit choice, rand(yesXurban yesXage yes) group(_id) id(district) ///
     draws(10000) burn(5000) saving(bdesh_draws) replace



Bayesian Mixed Logit Model                         Observations    =      3868
                                                   Groups          =        60
Acceptance rates:                                  Choices         =      1934
 Fixed coefs              =                        Total draws     =     10000
 Random coefs(ave,min,max)= 0.310, 0.231, 0.354    Burn-in draws   =      5000
-------------------------------------------------------------------------------
       choice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
Random        |
    yesXurban |   .7796584   .1905085     4.09   0.000     .4061781    1.153139
      yesXage |   .0067593   .0329065     0.21   0.837    -.0577519    .0712706
          yes |  -.7777077   .1154274    -6.74   0.000    -1.003996   -.5514193
--------------+----------------------------------------------------------------
Cov_Random    |
var_yesXurb

Suppose one wished to save some values of individual-level random parameters, but that the problem has too many individuals or requires too many draws to get to convergence. A useful approach in these circumstances is to complete
    a long first run without saving parameters, and then do a short second one using starting values. Suppose that the code in the previous example has been run. One can then run something to the effect of the following to get
    individual parameters:

In [7]:
mat b = e(b)
mat beta = b[1, 1..3]
mat V = b[1,4], b[1,5], b[1,6] \ b[1,5], b[1,7], b[1,8] \ b[1,6], b[1,7], b[1,9]

In [8]:
bayesmixedlogit choice, rand(yesXurban yesXage yes) group(_id) id(district) ///
     from(beta) fromv(V) draws(100) indsave(randpars) indkeep(50) replaceind



Bayesian Mixed Logit Model                         Observations    =      3868
                                                   Groups          =        60
Acceptance rates:                                  Choices         =      1934
 Fixed coefs              =                        Total draws     =       100
 Random coefs(ave,min,max)= 0.297, 0.160, 0.400    Burn-in draws   =         0
-------------------------------------------------------------------------------
       choice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
Random        |
    yesXurban |   .7763147   .1041361     7.45   0.000     .5697116    .9829178
      yesXage |   .0078787   .0344768     0.23   0.820    -.0605223    .0762797
          yes |  -.7578993   .1074017    -7.06   0.000    -.9709811   -.5448174
--------------+----------------------------------------------------------------
Cov_Random    |
var_yesXurb

One post-estimation idea is to get the mean for parameter values by individuals, and fit some kernel density to the means to view the distribution of the individual-level parameters:

In [1]:
bysort district: egen myesXurban = mean(yesXurban)
bysort district: gen last = _n == _N
kdensity myesXurban if last
graph display


no variables defined


r(111);
r(111);


### Example 3

Of course, it is possible to just do one run and retain all the information. As one final example:

In [ ]:
webuse union, clear
case2alt, casevars(age grade) choice(union) gen(unionmember)
bayesmixedlogit unionmember, rand(y0Xage y0Xgrade y0) group(_id) id(idcode) ///
     draws(1000) burn(800) saving(parm_draws) indsave(indparm_draws) indkeep(20) replace noisy


#### Stored results

**`bayesmixedlogit`** stores the following in e():

#### Scalars        

**`e`**(_`N`_)                number of observations

**`e`**(_`df_r`_)             degrees of freedom for summarizing draws (equal to number of retained draws)

**`e`**(_`krnd`_)             number of random parameters

**`e`**(_`kfix`_)             number of fixed parameters

**`e`**(_`draws`_)            number of draws

**`e`**(_`burn`_)            burn-in observations

**`e`**(_`thin`_)             thinning parameter

**`e`**(_`random_draws`_)     number of draws of each set of random parameters per pass

**`e`**(_`fixed_draws`_)      number of draws of fixed parameters per pass

**`e`**(_`damper_fixed`_)     damping parameter -- fixed parameters

**`e`**(_`damper_random`_)    damping parameter -- random parameters

**`e`**(_`opt_arate_fixed`_)  desired acceptance rate -- fixed parameters

**`e`**(_`opt_arate_random`_) desired acceptance rate -- random parameters

**`e`**(_`N_groups`_)         number of groups

**`e`**(_`N_choices`_)        number of choice occasions

**`e`**(_`arates_fa`_)        acceptance rate -- fixed parameters

**`e`**(_`arates_ra`_)        average acceptance rate -- random parameters

**`e`**(_`arates_rmax`_)      maximum acceptance rate -- random parameters

**`e`**(_`arates_rmin`_)      minimum acceptance rate -- random parameters

**`e`**(_`inddraws`_)         draws of individual parameters kept

#### Macros         

**`e`**(_`cmd`_)              **`bayesmixedlogit`**

**`e`**(_`depvar`_)           name of dependent variable

**`e`**(_`indepvars`_)        independent variables

**`e`**(_`title`_)            title in estimation output

**`e`**(_`properties`_)       b V

**`e`**(_`saving`_)           file containing results

**`e`**(_`fixed_sampler`_)    sampler type for fixed parameters

**`e`**(_`random_sampler`_)   sampler type for random parameters

**`e`**(_`random`_)           random parameter names

**`e`**(_`fixed`_)            fixed parameter names

**`e`**(_`identifier`_)       identifier for individuals

**`e`**(_`group`_)            identifier for choice occasions

**`e`**(_`indsave`_)          file holding individual-level parameter draws

#### Matrices       

**`e`**(_`b`_)                mean parameter values

**`e`**(_`V`_)                variance-covariance matrix of parameters

**`e`**(_`V_init`_)           initial variance-covariance matrix of random parameters

**`e`**(_`b_init`_)           initial mean vector of random parameters

**`e`**(_`arates_fixed`_)     row vector of acceptance rates of fixed parameters

**`e`**(_`arates_rand`_)      vector or matrix of acceptance rates of random parameters

#### Functions      

**`e`**(_`sample`_)           marks estimation sample



## Comments

The basic algorithms used in drawing are described in detail in [Baker(2014)](https://www.stata-journal.com/article.html?article=st0354).  The user might gain a fuller understanding of the options `aratefixed()`, `araterandom()`, `dampparmfixed()`, `dampparmrandom()`, and other options
    controlling adaptation of the proposal distribution from a reading of this document.

`bayesmixedlogit` requires that the package of `Mata` functions `amcmc()` be installed and also requires installation of Ben Jann's `moremata` set of extended `Mata` functions.

**Caution** -- While summary statistics of the results of a drawing are presented in the usual `Stata` format, `bayesmixedlogit` provides no guidance as to how one should actually select the number of draws, how one should process the
    draws, how one should monitor convergence of the algorithm, or how one should present and interpret results.  One would do well to consult [Train (2009)](https://eml.berkeley.edu/books/choice2.html) and a good source on Bayesian methods, such as [Gelman et al. (2009)](https://www.amazon.com/Bayesian-Analysis-Chapman-Statistical-Science/dp/158488388X).
    Fortunately, Stata provides many tools for summarizing and plotting the results of a drawing.


## References


Baker, M. J. 2014. _Adaptive Markov chain Monte Carlo sampling and estimation in `Mata`_. **Stata Journal** 14: 623-61.

Gelman, A., J. B. Carlin, H. S. Stern, and D. B. Rubin. 2009. **Bayesian data analysis**. 2nd. ed. Boca Raton, FL: Chapman & Hall/CRC.

Hole, A. R. 2007. _Fitting mixed logit models by using maximum simulated likelihood_. **Stata Journal** 7: 388-401.

Long, J. S., and J. Freese. 2006. **Regression Models for Categorical Dependent Variables Using Stata**. 2nd ed. College Station, TX: Stata Press

Train, K. E. 2009. **Discrete Choice Methods with Simulation**. 2nd ed. Cambridge: Cambridge University Press.
